In [5]:
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv 
import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [13]:
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

In [14]:
parent_path = '/home/tushar/Downloads/SVM_accuracy_final_new_'#

In [15]:
pred_col = 'pred'
actual_col = 'actual'

In [18]:
accuracy_df = pd.DataFrame()
for division in [1,2,3,4,5]:
    accuracy_df = accuracy_df.append(pd.read_pickle(parent_path+str(division)+'.pkl'))
accuracy_df = accuracy_df.rename(columns={'predicted_lable':'pred','true_label':'actual','words':'word','pred_tag':'pred','actual_tag':'actual'})


In [19]:
print(len(accuracy_df[accuracy_df[pred_col]==accuracy_df[actual_col]])/len(accuracy_df))

0.7170435594575225


In [20]:
unique_tags = accuracy_df[actual_col].unique()

In [21]:
unique_tags

array(['PRT', '.', 'ADV', 'ADJ', 'ADP', 'DET', 'NOUN', 'VERB', 'PRON',
       'CONJ', 'NUM', 'X'], dtype=object)

In [22]:
per_pos_df = pd.DataFrame()
for tag in unique_tags:
    temp_accuracy_df = accuracy_df[accuracy_df[actual_col]==tag]
    accuracy = len(temp_accuracy_df[temp_accuracy_df[pred_col]==temp_accuracy_df[actual_col]])/len(temp_accuracy_df)
    print(tag,len(temp_accuracy_df),accuracy)
    row = {}
    row['tag'] = tag
    row['count'] = len(temp_accuracy_df)
    row['accuracy'] = accuracy
    per_pos_df = per_pos_df.append(row,ignore_index=True)

PRT 29828 0.09075365428456483
. 147530 0.999972886870467
ADV 56232 0.042164603784322095
ADJ 83695 0.024314475177728657
ADP 144725 0.842881326653999
DET 136975 0.9629202409198759
NOUN 275475 0.9572883201742445
VERB 182712 0.46877052410350717
PRON 49326 0.8783805700847424
CONJ 38139 0.8025380843755736
NUM 14873 0.04020708666711491
X 1386 0.0


In [23]:
confusion_matrix = accuracy_df.groupby([pred_col,actual_col])['word'].count().unstack().fillna(0)

In [24]:
print(per_pos_df.round(2).to_latex())  

\begin{tabular}{lrrl}
\toprule
{} &  accuracy &     count &   tag \\
\midrule
0  &      0.09 &   29828.0 &   PRT \\
1  &      1.00 &  147530.0 &     . \\
2  &      0.04 &   56232.0 &   ADV \\
3  &      0.02 &   83695.0 &   ADJ \\
4  &      0.84 &  144725.0 &   ADP \\
5  &      0.96 &  136975.0 &   DET \\
6  &      0.96 &  275475.0 &  NOUN \\
7  &      0.47 &  182712.0 &  VERB \\
8  &      0.88 &   49326.0 &  PRON \\
9  &      0.80 &   38139.0 &  CONJ \\
10 &      0.04 &   14873.0 &   NUM \\
11 &      0.00 &    1386.0 &     X \\
\bottomrule
\end{tabular}



In [25]:
confusion_matrix = confusion_matrix.iloc[1:]

In [26]:
i = 0
prec_recall = pd.DataFrame()
for idx in confusion_matrix.index:
    row = {}
    row['tag'] = idx
    row['recall'] = rec = recall(i,np.array(confusion_matrix).T)
    row['precision'] = prec = precision(i,np.array(confusion_matrix).T)
    row['F1 score'] = 2*(rec * prec) / (rec + prec)


    i+=1
    prec_recall = prec_recall.append(row,ignore_index=True)
    

/home/tushar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [27]:
print(confusion_matrix.astype(int).to_latex(index=True))  

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
actual &  . &    ADJ &     ADP &    ADV &   CONJ &     DET &    NOUN &   NUM &   PRON &    PRT &   VERB &     X \\
pred &    &        &         &        &        &         &         &       &        &        &        &       \\
\midrule
ADJ  &  0 &   2035 &       0 &   1755 &      0 &       0 &     340 &   303 &      0 &    105 &    330 &     0 \\
ADP  &  0 &   2284 &  121986 &   5401 &   3489 &     494 &     383 &     0 &    224 &  15119 &   1031 &    20 \\
ADV  &  0 &    720 &     368 &   2371 &      0 &       0 &      25 &     0 &      0 &      0 &      0 &     0 \\
CONJ &  0 &      0 &     525 &     11 &  30608 &       0 &       0 &     0 &      0 &      0 &      0 &     4 \\
DET  &  0 &   1862 &    5679 &   1161 &     12 &  131896 &     402 &  1750 &   2864 &    745 &     65 &    20 \\
NOUN &  4 &  73898 &    7957 &  27875 &    449 &    2399 &  263709 &  9341 &   2008 &   3485 &  92930 &  1212 \\
NUM  &  0 &      0 &       0 &      0 &      

In [28]:
print(prec_recall.round(2).to_latex(index=True))  

\begin{tabular}{lrrrl}
\toprule
{} &  F1 score &  precision &  recall &   tag \\
\midrule
0 &       NaN &       0.00 &    0.00 &   ADJ \\
1 &      0.02 &       0.02 &    0.03 &   ADP \\
2 &      0.01 &       0.11 &    0.00 &   ADV \\
3 &      0.00 &       0.00 &    0.00 &  CONJ \\
4 &      0.00 &       0.00 &    0.00 &   DET \\
5 &      0.01 &       0.00 &    0.02 &  NOUN \\
6 &       NaN &       0.00 &    0.00 &   NUM \\
7 &      0.02 &       0.02 &    0.07 &  PRON \\
8 &       NaN &       0.00 &    0.00 &   PRT \\
9 &      0.01 &       0.01 &    0.03 &  VERB \\
\bottomrule
\end{tabular}



In [29]:
prec_recall.mean()

F1 score     0.010063
precision    0.014947
recall       0.014750
dtype: float64